In [3]:
import mediapipe as mp
import cv2
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import math
import numpy as np
import time

prev_x, prev_y = None, None

prev_finger_x = None




# STEP 2: Create a HandLandmarker object.
base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_options, num_hands=2)
detector = vision.HandLandmarker.create_from_options(options)

# Function to draw landmarks with numbers and connect them with lines.
def draw_landmarks_on_image(image, detection_result):
    annotated_image = image.copy()

    # Define the connections between landmarks (e.g., thumb to index, etc.)
    connections = [
        (0, 1), (1, 2), (2, 3), (3, 4),  # Thumb
        (5, 6), (6, 7), (7, 8),  # Index finger
        (9, 10), (10, 11), (11, 12),  # Middle finger
        (13, 14), (14, 15), (15, 16),  # Ring finger
        (17, 18), (18, 19), (19, 20)  # Pinky finger
    ]

    # Loop over detected hands
    for hand_world_landmarks in detection_result.hand_landmarks:
        # Loop over each landmark in the hand.
        for i, landmark in enumerate(hand_world_landmarks):
            x = int(landmark.x * annotated_image.shape[1])
            y = int(landmark.y * annotated_image.shape[0])
            cv2.circle(annotated_image, (x, y), 5, (0, 255, 0), -1)  # Green circle for each landmark
            cv2.putText(annotated_image, str(i), (x + 10, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)  # Red text for the landmark number

        # Draw lines between the landmarks
        for connection in connections:
            start_idx, end_idx = connection
            if start_idx < len(hand_world_landmarks) and end_idx < len(hand_world_landmarks):
                start_landmark = hand_world_landmarks[start_idx]
                end_landmark = hand_world_landmarks[end_idx]
                start_x = int(start_landmark.x * annotated_image.shape[1])
                start_y = int(start_landmark.y * annotated_image.shape[0])
                end_x = int(end_landmark.x * annotated_image.shape[1])
                end_y = int(end_landmark.y * annotated_image.shape[0])
                cv2.line(annotated_image, (start_x, start_y), (end_x, end_y), (255, 0, 0), 2)  # Blue lines between landmarks

    return annotated_image



# STEP 3: Open the webcam.
cap = cv2.VideoCapture(0)  # Use 0 for default webcam

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# STEP 4: Loop over the webcam frames and detect hand landmarks.
while cap.isOpened():
    ret, frame = cap.read()
    frame = cv2.flip(frame,1)
    if not ret:
        print("Error: Failed to capture image.")
        break
    
    # Convert the frame from BGR (OpenCV format) to RGB for Mediapipe.
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)

    # Detect hand landmarks in the frame.
    detection_result = detector.detect(mp_image)


        
    # STEP 5: Process the classification result and visualize it.
    if detection_result.hand_landmarks:  # Check if hand landmarks are detected
        annotated_frame = draw_landmarks_on_image(frame, detection_result)
                        
          
    else:
        annotated_frame = frame

    


    # annotated_frame =cv2. flip(annotated_frame,1)
    # drawing_image = cv2.flip(drawing_image,1)
    # frame = cv2.flip(frame,1)


    # Show the frame with hand landmarks annotated.

    cv2.imshow("Hand Landmarks", annotated_frame)


    # Exit when 'q' is pressed.
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
   # STEP 6: Release resources.
cap.release()
cv2.destroyAllWindows()

c:\Users\samma\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
